In [119]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np
from sklearn.model_selection import train_test_split

# from sklearn.model_selection import train_test_split

# Load datasets

X_train = pd.read_parquet(
    r"D:\Duke\IDS705 ML\Final_project\enivornmental_impact_of_aviation\Train-test-validation_with_outliers/X_train.parquet"
)
X_test = pd.read_parquet(
    r"D:\Duke\IDS705 ML\Final_project\enivornmental_impact_of_aviation\Train-test-validation_with_outliers/X_test.parquet"
)
X_val = pd.read_parquet(
    r"D:\Duke\IDS705 ML\Final_project\enivornmental_impact_of_aviation\Train-test-validation_with_outliers/X_val.parquet"
)
y_train = pd.read_parquet(
    r"D:\Duke\IDS705 ML\Final_project\enivornmental_impact_of_aviation\Train-test-validation_with_outliers/y_train.parquet"
).squeeze()
y_test = pd.read_parquet(
    r"D:\Duke\IDS705 ML\Final_project\enivornmental_impact_of_aviation\Train-test-validation_with_outliers/y_test.parquet"
).squeeze()
y_val = pd.read_parquet(
    r"D:\Duke\IDS705 ML\Final_project\enivornmental_impact_of_aviation\Train-test-validation_with_outliers/y_val.parquet"
).squeeze()

In [120]:
df = pd.read_parquet(
    r"D:\Duke\IDS705 ML\Final_project\enivornmental_impact_of_aviation\cleaned_aviation_data_with_outliers_v4.parquet"
)

In [ ]:
X = df[
    [
        "seats",
        "acft_class",
        "n_flights",
        "departure_continent",
        "arrival_continent",
        "departure_country",
        "arrival_country",
        "domestic",
        "ask",
        "rpk",
        "fuel_burn",
    ]
]
y = df["co2_per_distance"]

# One-hot encode categorical variables

### Adding new features

In [124]:
X.loc[:, "same_continent"] = X["arrival_country"] == X["departure_country"]
X.loc[:, "fuel_burn_per_seat"] = X["fuel_burn"] / X["seats"]
X.loc[:, "ask_per_seat"] = X["ask"] / X["seats"]
X.loc[:, "ask_per_flight"] = X["ask"] / X["n_flights"]
X.loc[:, "is_widebody"] = X["acft_class"].apply(lambda x: 1 if x == "WB" else 0)
X.loc[:, "is_narrowbody"] = X["acft_class"].apply(lambda x: 1 if x == "NB" else 0)

C:\Users\Afag\AppData\Local\Temp\ipykernel_17416\569032425.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.loc[:, "same_continent"] = X["arrival_country"] == X["departure_country"]
C:\Users\Afag\AppData\Local\Temp\ipykernel_17416\569032425.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.loc[:, "fuel_burn_per_seat"] = X["fuel_burn"] / X["seats"]
C:\Users\Afag\AppData\Local\Temp\ipykernel_17416\569032425.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFra

In [125]:
X = pd.get_dummies(X, drop_first=True)

In [128]:
# Split into train + validation and test sets (80% train_val, 20% test)
X_train_val, X_test, y_train_val, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Split train + validation into separate train and validation sets (now 64% train, 16% val)
X_train, X_val, y_train, y_val = train_test_split(
    X_train_val, y_train_val, test_size=0.2, random_state=42
)

In [130]:
X_train.to_parquet(
    r"D:\Duke\IDS705 ML\Final_project\enivornmental_impact_of_aviation\Train-test-validation_with_outliers/X_train.parquet"
)
X_test.to_parquet(
    r"D:\Duke\IDS705 ML\Final_project\enivornmental_impact_of_aviation\Train-test-validation_with_outliers/X_test.parquet"
)
X_val.to_parquet(
    r"D:\Duke\IDS705 ML\Final_project\enivornmental_impact_of_aviation\Train-test-validation_with_outliers/X_val.parquet"
)
y_test.to_frame(name="co2_per_distance").to_parquet(
    r"D:\Duke\IDS705 ML\Final_project\enivornmental_impact_of_aviation\Train-test-validation_with_outliers/y_test.parquet"
)

y_train.to_frame(name="co2_per_distance").to_parquet(
    r"D:\Duke\IDS705 ML\Final_project\enivornmental_impact_of_aviation\Train-test-validation_with_outliers/y_train.parquet"
)
y_val.to_frame(name="co2_per_distance").to_parquet(
    r"D:\Duke\IDS705 ML\Final_project\enivornmental_impact_of_aviation\Train-test-validation_with_outliers/y_val.parquet"
)
# X_train.to_parquet(r"D:\Duke\IDS705 ML\Final_project\enivornmental_impact_of_aviation\Train-test-validation_with_outliers/X_train.parquet")

In [95]:
X_train

,distance_km,co2,seats,acft_class,n_flights,departure_continent,departure_country,arrival_continent,arrival_country,domestic,ask,rpk,fuel_burn
58430,2807.618187,1.470188e+06,10465.000000,NB,57.5,NA,US,NA,US,1,2.938172e+07,2.421054e+07,4.652493e+05
14544,585.271316,2.767955e+05,2896.000000,TP,362.0,NA,US,NA,US,1,1.694946e+06,1.396635e+06,8.759352e+04
1846,195.953331,1.654228e+06,84480.714650,TP,1204.5,EU,DK,EU,DK,1,1.655428e+07,1.364072e+07,5.234898e+05
54864,512.719083,1.814154e+05,4734.286624,TP,67.5,AF,EH,AF,MA,0,2.427359e+06,2.000144e+06,5.740993e+04
167771,1403.154844,2.356261e+04,249.093750,NB,1.5,AF,MA,EU,FR,0,3.495171e+05,2.880021e+05,7.456522e+03
...,...,...,...,...,...,...,...,...,...,...,...,...,...
279080,1450.536982,8.519297e+05,7537.253500,None,4.0,AS,IN,AS,MM,0,1.093306e+07,9.008846e+06,2.695980e+05
132911,294.038697,2.161344e+04,586.404777,NB,3.0,EU,CH,EU,DE,0,1.724257e+05,1.420788e+05,6.839698e+03
198702,7859.794833,6.117166e+04,14.000000,PJ,1.0,NA,US,EU,CH,0,1.100371e+05,9.067059e+04,1.935812e+04
30457,819.145991,9.376916e+05,13169.761510,TP,174.0,EU,AT,EU,CH,0,1.078796e+07,8.889277e+06,2.967378e+05


In [ ]:
# Identify categorical columns
categorical_cols = X_train.select_dtypes(
    include=["object", "category"]
).columns.tolist()

In [50]:
rf_pipeline = Pipeline(
    steps=[
        # ("preprocessor", preprocessor),
        ("regressor", RandomForestRegressor(n_estimators=100, random_state=42)),
    ]
)

In [51]:
rf_pipeline.fit(X_train, y_train)

# Predict on test set
y_pred_test = rf_pipeline.predict(X_test)

In [ ]:
# Evaluate on test set
r2 = r2_score(y_test, y_pred_test)
mse = mean_squared_error(y_test, y_pred_test)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, y_pred_test)

print(f"R-squared (R²): {r2:.4f}")
# print(f"Mean Squared Error (MSE): {mse:.4f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.4f}")
print(f"Mean Absolute Error (MAE): {mae:.4f}")

# Predict and evaluate on validation set
y_pred_val = rf_pipeline.predict(X_val)

r2_val = r2_score(y_val, y_pred_val)
mse_val = mean_squared_error(y_val, y_pred_val)
rmse_val = np.sqrt(mse_val)
mae_val = mean_absolute_error(y_val, y_pred_val)

print("\nValidation Set Performance:")
print(f"R-squared (R²): {r2_val:.4f}")
# print(f"Mean Squared Error (MSE): {mse_val:.4f}")
print(f"Root Mean Squared Error (RMSE): {rmse_val:.4f}")
print(f"Mean Absolute Error (MAE): {mae_val:.4f}")

R-squared (R²): 0.9700
Root Mean Squared Error (RMSE): 974.6698
Mean Absolute Error (MAE): 131.3191

Validation Set Performance:
R-squared (R²): 0.9734
Root Mean Squared Error (RMSE): 883.8049
Mean Absolute Error (MAE): 126.1205
